__Overview__:

We start with the wrangled data on US regional transit lines, containing fields of general description, numerical summary data, and monthly time series data on four different values: unlinked passenger trips (UPT), vehicle revenue miles (VRM), vehicle revenue hours (VRH), and vehicles operated in annual maximum service (VOMS). This consists of 42 out of the 2,237 original records, and 1,028 fields (all 1,026 original fields, plus 2 that were created to improve analysis). Many of the original records were removed because they either were missing too much data, were determined to be insignificant to an area compared to the other lines, or otherwise didn't meet some criterion for the purposes of this study. Others were subsequently removed due to the sheer number of lines that were still eligible for modeling, and therefore they were limited to the ones determined most likely to project the results we are looking for.

The purpose of this step is to prepare the data for modeling.

Goals for this step:

 - Consider if dummy/indicator features should be created from categorical variables.
 - Consider standardizing numeric features.
 - Consider any other transformations.

In [7]:
# Import all necessary packages.

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sb_utils import save_file

__Objective__: Load the data.

In [8]:
pd.set_option('display.max_columns', None)

In [9]:
# Import and display the cleaned transit data.
# Raw data source: https://www.transit.dot.gov/ntd/data-product/monthly-module-raw-data-release
# Glossary of field names and values: https://www.transit.dot.gov/ntd/national-transit-database-ntd-glossary

transit_data = pd.read_csv('transit_data_explored.csv')
transit_data

5 digit NTD ID 4 digit NTD ID  \
0               40             40   
1            10017           1017   
2            10040           1040   
3            10049           1049   
4            10053           1053   
5            10056           1056   
6            20169           2169   
7            20175           2175   
8            20217           2217   
9            30006           3006   
10           30058           3058   
11           30072           3072   
12           30087           3087   
13           40002           4002   
14           40009           4009   
15           40046           4046   
16           40058           4058   
17           40108           4108   
18           40192           4192   
19           41105       4R04-020   
20           50003           5003   
21           50017           5017   
22           50039         5039.0   
23           50149         5149.0   
24           50157         5157.0   
25           50169         5169.0   
26           50198         5198.0   
27           50199         5199.0   
28           50204         5204.0   
29           50518         5221.0   
30           60008         6008.0   
31           60059         6059.0   
32           60102         6102.0   
33           60107         6107.0   
34           60130         6130.0   
35           70044            NaN   
36           70049         7049.0   
37           80012         8012.0   
38           90026         9026.0   
39           90032         9032.0   
40           90140         9140.0   
41           90157         9157.0   

                                               Agency Modes TOS  Active  \
0      Central Puget Sound Regional Transit Authority    LR  DO  Active   
1                   Greater Hartford Transit District    DR  PT  Active   
2                     Southeast Area Transit District    MB  DO  Active   
3                  Greater New Haven Transit District    DR  DO  Active   
4                   Cape Ann Transportation Authority    DR  PT  Active   
5   Connecticut Department of Transportation - CTT...    MB  DO  Active   
6                            Trans-Bridge Lines, Inc.    CB  DO  Active   
7                  Private Transportation Corporation    MB  PT  Active   
8                                Hampton Jitney, Inc.    CB  DO  Active   
9                    Greater Richmond Transit Company    MB  DO  Active   
10                                    City of Fairfax    MB  DO  Active   
11                         Frederick County, Maryland    MB  DO  Active   
12                                  County of Fayette    DR  DO  Active   
13                                  City of Knoxville    MB  DO  Active   
14                               City of Fayetteville    MB  DO  Active   
15                                   Sarasota County     DR  PT  Active   
16                                      City of Rome     DR  DO  Active   
17  Research Triangle Regional Public Transportati...    MB  DO  Active   
18                                      Martin County    MB  PT  Active   
19              Audubon Area Community Services, Inc.    DR  DO  Active   
20                                    City of Kenosha    MB  DO  Active   
21          Greater Dayton Regional Transit Authority    TB  DO  Active   
22         Saginaw Transit Authority Regional Service    DR  DO  Active   
23               Michiana Area Council of Governments    MB  PT  Active   
24           Butler County Regional Transit Authority    DR  DO  Active   
25                                Miami County, Ohio     DR  PT  Active   
26                       Medina County Public Transit    MB  DO  Active   
27                      Delaware County Transit Board    DR  DO  Active   
28               Jackson County Mass Transit District    MB  DO  Active   
29                                  SouthWest Transit    DR  PT  Active   
30  Metropolitan Transit Authority of Harris Count...    MB  PT  Active   
31          

In [10]:
transit_data.describe()

5 digit NTD ID         UZA  UZA Area SQ Miles  UZA Population  \
count       42.000000   42.000000          42.000000    4.200000e+01   
mean     45120.904762  136.309524         701.500000    2.670892e+06   
std      23710.095683  151.721260         919.843949    4.911122e+06   
min         40.000000    1.000000          30.000000    5.137000e+04   
25%      30061.500000   15.250000          71.500000    1.420800e+05   
50%      50010.000000   61.500000         351.000000    6.836755e+05   
75%      60046.250000  229.250000         954.500000    2.880282e+06   
max      90157.000000  487.000000        3450.000000    1.835130e+07   

       Service Area SQ Miles  Service Area Population  \
count              42.000000             4.200000e+01   
mean             1233.119048             1.094474e+06   
std              2868.583267             2.378480e+06   
min                 6.000000             2.256500e+04   
25%                83.250000             1.335265e+05   
50%               330.000000             2.003165e+05   
75%               789.000000             5.506328e+05   
max             15355.000000             1.163811e+07   

       Most Recent Report Year  FY End Month  FY End Year  Passenger Miles FY  \
count                     42.0     42.000000         42.0        4.200000e+01   
mean                    2021.0      8.404762       2021.0        1.321883e+07   
std                        0.0      2.669242          0.0        3.056461e+07   
min                     2021.0      6.000000       2021.0        7.615300e+04   
25%                     2021.0      6.000000       2021.0        3.930340e+05   
50%                     2021.0      7.000000       2021.0        1.231873e+06   
75%                     2021.0     12.000000       2021.0        7.126938e+06   
max                     2021.0     12.000000       2021.0        1.347366e+08   

       Unlinked Passenger Trips FY  Unlinked Passenger Trips UZA Total  \
count                 4.200000e+01                        4.200000e+01   
mean                  2.342751e+06                        4.025786e+06   
std                   6.737262e+06                        1.071048e+07   
min                   1.493200e+04                        1.493200e+04   
25%                   5.654150e+04                        1.030990e+05   
50%                   2.240450e+05                        2.847905e+05   
75%                   1.109066e+06                        1.569988e+06   
max                   3.841412e+07                        4.423402e+07   

       Unlinked Passenger Trips UZA Perc  Average Trip Length FY  \
count                          42.000000               42.000000   
mean                           76.362847               10.067564   
std                            33.164649               15.616546   
min                            10.514473                0.900000   
25%                            54.326213                4.841500   
50%                            97.382452                6.220550   
75%                           100.000000                8.873625   
max                           100.000000               88.145500   

           Fares FY  Operating Expenses FY  Average Cost per Trip FY  \
count  4.200000e+01           4.200000e+01                 42.000000   
mean   2.129105e+06           2.054506e+07                 29.440076   
std    4.629890e+06           4.135143e+07                 22.079430   
min    3.409000e+03           8.309720e+05                  2.781700   
25%    7.130800e+04           1.991275e+06                 10.225925   
50%    4.342825e+05           3.990116e+06                 26.404000   
75%    1.187603e+06           1.508038e+07                 43.668700   
max    1.933750e+07           1.796938e+08                 79.614900   

       Average Fares per Trip FY     UPT_JAN02     UPT_FEB02     UPT_MAR02  \
count                  42.000000  1.700000e+01  1.700000e+01  1.700000e+01   
mean                    5.053

__Objective__: Consider creating dummy/indicator variables.

In [11]:
# There are no categorical variables, so no dummy/indicator variables will be created.

__Objective__: Consider standardizing numeric features.

In [12]:
# We want to standardize each line's historical UPT data by dividing each month's value by the respective maximum UPT between January 2017 and December 2019.
# We also need to record each line's maximum UPT during this time so that the data can be untransformed later.
# We can do so by storing them in a dictionary, but since the dictionary can't be carried across notebooks, this must be done in the modeling workbook instead of here.
# The standardization will therefore also be done in the modeling workbook.

__Objective__: Consider any other transformations.

In [15]:
# - The data will need to be split into training and test sets, but first requires some rearrangement that must be done in the modeling workbook instead of here.
#   The split will therefore also be done in the modeling workbook.
# - The VRM, VHS, and VOMS fields have not been used so far in this study. They will be kept in the data for now but we may consider dropping them later to tidy up the data.

In [16]:
# Save the preprocessed data as a new file (no changes were actually made).

save_file(transit_data, 'transit_data_preprocessed.csv', '../Capstone 3')

A file already exists with this name.

Writing file.  "../Capstone 3\transit_data_preprocessed.csv"
